In [5]:
from sympy import symbols, Eq, solve
import math

#A charge of compounds dictionary is needed for the charge 

#If the salts of strong acids and bases have been used in the solution, such as NaCl, 
#the concentrations of compounds Na and Cl coming from this salt have to be removed from trad_salt_elements as they don't contribute to the pH.
# They also won't contribute to charge balance as the two charged compounds from the salt will cancel out. 


"retrieve Ka value" 
def pH (concentration_of_compounds_in_solution, temperature):
    #setting index for compounds possesing pKa values
    i=0
    #setting index for compounds possesing Ksp values (metals)
    l=0
    #setting variable of the concentration of protons in solution
    H_plus_concentration=symbols("H_plus_concentration")
    #setting variable of the concentration of OH- in solution 
    OH_minus_concentration=symbols("OH_minus_concentration")
    
    # Defines system of equations
    equations = []
    
    # Initialize a variable to accumulate known values for charge balance calculations 
    accumulated_known_values_for_charge_balance = 0
    # Initialize a variable to accumulate variables for charge balance calculations 
    accumulated_variables_for_charge_balance = 0
    # Determines the ionization constant of water (Kw) at a specific temperature
    Kw=Value_of_Ionisation_Constant(temperature)
    # Defines the equation for the ionization constant of water
    Kw_equation=eq(H_plus_concentration*OH_minus_concentration,Kw)
    equations.append(Kw_equation)
    #Adding OH_minus and H_plus to charge balance
    accumulated_variables_for_charge_balance+=H_plus_concentration-OH_minus_concentration 

    for compound, compound_concentration in concentration_of_compounds_in_solution:
        #Below, it is checked as to whether or not the compound has a solubility product constant or an acid dissociation constant. The former allows one to define the impact that metal ions have on the pH of the solution
        #One can see Ksp having an impact on the pH with the example that follows. We have an initial salt we place in a solution, i.e, CaSO4, which then dissociates into Ca^2+ and SO4^2-. 
        #There is then a Ksp constant that depends on the concentration of Ca^2+, OH^-1 and Ca(OH)2 and a Ka value that depends on the concentration of SO4^2-, HSO4^1- and H2SO4 as well as H+
        #The Ksp constant has an impact on the concentration of OH^-1 which in itself impact the pH of the solution.
        if compound in Ksp_values:
            #Defines a constant for the initial concentration of the compound
            globals()[f'Ksp_value_of_compound_{l}']=Ksp_values[compound]
            #Defines variables for the concentrations of the metal in a dissociated state or not, i.e, Ca(OH)^2- or Ca^2+
            exec("dissociated_metal_{}_concentration=symbols('dissociated_metal_{}_concentration')".format(l,l))
            exec("undissociated_metal_compound_{}_concentration=symbols('undissociated_metal_compound_{}_concentration')".format(l,l))
            #Equation of dissociation equilibrium
            exec(f"globals()['equation_Ksp_{l}'] = Eq(({OH_minus_concentration}*['dissociated_metal_{l}_concentration'])/['undissociated_metal_{l}_concentration'], globals()['Ksp_value_of_compound_{l}'])")
            equations.append(f'equation_Ksp_{l}')
            # Accumulating variables for charge balance. 
            accumulated_variables_for_charge_balance+=compound_charge[compound]*f'dissociated_metal_{l}_concentration'
            #Mass equation of metal/metal compound
            exec(f"mass_equation_metal_{l} = Eq(dissociated_metal_{l}_concentration + undissociated_metal_{l}_concentration, compound_concentration)")
            equations.append(f'mass_equation_metal_{l}')
            l+=1
                
        else: 
            #Defines the constant that is the initial concentration of the compound 
            globals()[f'initial_compound_concentration_{i}']=compound_concentration 
            #Initialize a variable to accumulate the variables of the concentrations into one equation of mass balance
            accumulated_variables_for_mass_balance=0
            #Determines the number of states a compound is in whose concentrations define one of the known Ka values of the compound, there could be more states, but they are negligeable. 
            number_of_states_of_dissociation=len(pKa_values[compound])
            if number_of_states_of_dissociation!=1:
                for s in range (number_of_states_of_dissociation):
                    #derives the Ka_th value of a compound
                    globals()[f'Ka_{s}_value_of_compound_{i}']=float(10**(-pKa_values[compound][s]))
                    #This line below generates symbolic variables representing the concentrations of 
                    #different forms of a compound, where each variable corresponds to a state of dissociation "s" of compound "i" .
                    exec("A_{}_{}_concentration= symbols('A_{}_{}_concentration')".format(s,i,s,i))
                    # Accumulating variables for charge balance. Here s is used as a substitute for the charge as we have multiple charges which depend on the state of the compound.
                    accumulated_variables_for_charge_balance+=(highest_existing_charge_of_compounds[compound]-s)*f'A_{s}_{i}_concentration'
                for s in range(number_of_states_of_dissociation):
                    accumulated_variables_for_mass_balance+=f'A_{s}_{i}_concentration'
                    if s!=number_of_states_of_dissociation:
                        exec(f"globals()['equation_Ka_{s}_{i}'] = Eq(({H_plus_concentration}*['A_{s+1}_{i}_concentration'])/['A_{s}_{i}_concentration'],[Ka_{s}_value_of_compound_{i}])")
                        equations.append(f'equation_Ka_{s}_{i}')
                mass_equation_i=Eq(accumulated_variables_for_mass_balance,compound_concentration)
                equations.append(mass_equation_i)
                i+=1
    Charge_Balance_equation= Eq(accumulated_variables_for_charge_balance, 0)
    equations.append(Charge_Balance_equation)
    solution = solve(equations, (H_plus_concentration, OH_minus_concentration))
    pH=-math.log(solution[0])
    return pH 



Test_concentration_of_solution = {
    "Ca": 0.036,
    "Mg": 0.013,
    "K": 0.296,
    "PO4": 0.026,
    "BO3":0.050,
    "Mn": 0.0003,
    "Zn": 0.0003,
    "Cu": 0.00005,
    "Fe(3+)": 0.0012,
    "NO3": 0,
    "SO4": 0,
    "Cl" : 0,
    "NH4" : 0,}

Ksp_values={
    "Ca": 7.9*10**(-6),
    "Mg": 1.5*10**(-11),
    "K": 10**4, #Complete dissociation of KOH 
    "Mn": 4.6*10**(-14),
    "Zn": 4.5*10**(-17),
    "Cu": 1.6*10**(-19),
    "Fe(3+)": 6.3*10**(-38),}

pKa_values={
    "PO4": [2.12,7.21,12.32],
    "BO3":[9.24,12.4,13.3],
    "NO3":[-1.4],
    "SO4": [-3,1.9],
    "Cl" : 0,
    "NH4" : 0,}

In [8]:
from sympy import symbols, Eq, solve

globals()[f'compound_concentration_{3}']=4
compound_concentration_3


4

In [9]:
from sympy import symbols

# Define the number of variables
n = 5  # You can change this to the desired number
variables=0
# Define the symbols for the variables using a loop
for i in range(1, n+1):
    exec("C{} = symbols('C{}')".format(i, i))

eq1 = Eq(2*C1 + 3*C2, 8)
eq2 = Eq(C1 - 2*C2, 1)
solution = solve((eq1, eq2), (C1, C2))

print("Solution:", solution)


Solution: {C1: 19/7, C2: 6/7}


In [10]:
from sympy import symbols, Eq

# Define symbols
x, y = symbols('x y')

# Define coefficients
coefficients = [(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]
constants = [10, 20, 30, 40, 50]

# Create equations and assign them to dynamically named variables
for i, (a, b) in enumerate(coefficients, start=1):
    eq_name = f"equation_{i}"
    c = constants[i-1]
    globals()[eq_name] = Eq(a * x + b * y, c)

# Now you have equations named equation_1, equation_2, ..., equation_5
# You can directly access them by their names
print(equation_1)
print(equation_2)
# And so on...
solution = solve((equation_1, equation_2), (x, y))
print("Solution:", solution)


Eq(x + 2*y, 10)
Eq(3*x + 4*y, 20)
Solution: {x: 0, y: 5}


In [30]:
# Dictionary of pKa values
pKa_values = {
    "Acetic Acid": [4.76],
    "Ammonium": [9.25],
    "Phenol": [9.95, 19.0],
    "Carbonic Acid": [6.37, 10.25]
}

# Accessing pKa values
print("pKa values of Acetic Acid:", pKa_values["Acetic Acid"][0])
print("pKa values of Phenol:", pKa_values["Phenol"])


pKa values of Acetic Acid: 4.76
pKa values of Phenol: [9.95, 19.0]


In [47]:
s=1
i=3
exec("A_{}_concentration_{} = symbols('A_{}_concentration_{}')".format(s,i,s,i))
equation_1=Eq(A_1_concentration_3,5)
solution=solve(equation_1,(A_1_concentration_3))
print(solution)


[5]


In [51]:
p=[0,4,1]

for m in range(len(p)): 
    print(m)

0
1
2


In [6]:
my_dict = {'a': 1, 'b': 2, 'c': 3}

element_to_check = 'd'

if element_to_check in my_dict:
    print(f"{element_to_check} exists in the dictionary.")
else:
    print(f"{element_to_check} does not exist in the dictionary.")


d does not exist in the dictionary.
